In [ ]:
from faker import Faker
import pandas as pd
import random
import os

fake = Faker(locale='en_US')      #instance of Faker for US addresses
fake_latam= Faker(locale='es_AR') #instance of Faker for Latin American addresses

Addresses_list = []  
Customers_list=[]
Payments_list=[]
Orders_list=[]

def addNewRecordToPaymentsTable(Payments_list,payment_id,customer_id):
    #for i in range(1,num_of_payments): #len(Payments_list)=998 #customers_list index's are in range 0-997 
    print("Inside addNewRecordToPaymentsTable")
    Payment={}
   
    Payment["payment_id"]=payment_id
    Payment["customer_id"]=customer_id
        #according to cafe rules
        #payments can be created only by employees with the position cashier and waiter
        #emp_id's of all cashiers and waiters are:16,17,18,19,20,26,27,28,29,30             
    Payment["emp_id"]=  fake.random_element(elements=(16,16,16,17,17,18,19,20,26,27,28,29,29,30,30))
    Payment["payment_date"]=str(fake.date_between_dates("-5y","+1w"))+" "+str(fake.time(pattern="%X")) 
    Payment["amount"]= random.randrange(1, 30, 2)*random.randrange(1, 10, 3)*0.99
    Payment["payment_type"]=fake.random_element(elements=("visa","visa","visa","visa","master card","master card","amex","cash","cash","pay pal",))
    Payments_list.append(Payment)
    print("payment for order_id was created")        
    
    

def addMoreOrdersToOrdersTable(Orders_list,num_of_orders):
    item_id_test=0
   
    for i in range(1000, 1000+num_of_orders):
        Order={}
        Order["order_id"]=i
        Order["item_id"]=fake.random_element(elements=(2,2,2,2,4,4,4,4,1,1,1,3,3,3,5,5,6,7,7)) 
        
        Order["customer_id"]=random.choice(Customers_list)["customer_id"]
        print(Order["customer_id"])
        

        Order["cafe_id"]=fake.random_element(elements=(1,2,3,4,5))
        Order["payment_id"]=i
        
        Order["delivery"]=fake.random_element(elements=("FALSE","FALSE","FALSE","FALSE","TRUE"))
        addNewRecordToPaymentsTable(Payments_list,Order["payment_id"],Order["customer_id"])
      
        Order["delivery_address_id"]=random.choice(Customers_list)["address_id"]
        Order["created_at"]=str(fake.date_between_dates("-5y","+1w"))+" "+str(fake.time(pattern="%X"))
        Order["item_quantity"]=fake.random_element(elements=(1,1,1,1,1,1,1,1,2,2,2,2,2,3,3,4,5,6))
        Order["customer_modifications"] =fake.random_element(elements=(" "," "," "," ","Add extra utensils please",
                                                                       "No utensils"))
        if  Order["delivery"]=="TRUE":
         Order["delivery_notes"] =fake.random_element(elements=(" "," ","Leave at the reception",
                                                                "Contactless delivery.Leave at my doorsteps",
                                                                "Please dont call.Text instead"))                                                    
                                                 
        if  Order["delivery"]=="FALSE": 
         Order["delivery_notes"] =" "  
        
        
        Orders_list.append(Order)
    
 
  
    # test for # addNewItemToExistingOrder(Orders_list,1,8,77)
    # test for # test updateItemQuantity(Orders_list,1, 10, 79)

    #second iteration:adding more items to random orders:
    
    addOneMoreItemLineToRandomExistingOrders(Orders_list)
    addOneMoreItemLineToRandomExistingOrders(Orders_list)
    return pd.DataFrame(Orders_list)
    return pd.DataFrame(Payments_list)
  
def addNewItemToExistingOrder(Orders_list,order_id,item_id,qty):
    print("Inside addNewItemToExistingOrder")
    items_list=[] 
    order_id_exists="false"
    item_id_exists="false"
    
    new_order_line={}  
    
    #verifying that this order with order_id was already created
    for order in Orders_list:
        if order.get("order_id") == order_id:
            items_list.append(order["item_id"])
            order_id_exists="true"          
            new_order_line["order_id"]=order["order_id"]  
            new_order_line["customer_id"]=order["customer_id"]   
            new_order_line["cafe_id"]=order["cafe_id"]
            new_order_line["payment_id"]=order["payment_id"]
            new_order_line["delivery"]=order["delivery"]
            new_order_line["delivery_address_id"]=order["delivery_address_id"]
            new_order_line["created_at"]=order["created_at"]
            new_order_line["item_quantity"]=order["item_quantity"]  
            new_order_line["customer_modifications"]=order["customer_modifications"]
            new_order_line["delivery_notes"]=order["delivery_notes"]
                  
    if order_id_exists=="false":
        print("Order with order_id "+ str(order_id)+" was not created yet")
        return False
        
    #now lets verify that order with order_id doesn't have item with requested item_id yet
    elif order_id_exists=="true":
        for item_ in items_list:
            if item_==item_id:
                item_id_exists="true"
                print("Order with order_id "+ str(order_id)+" already contains item "+str(item_id))
                return False
     # if we successfully verified that order with order_id already exists 
     # and it doesn't contain item with requested item_id
     # then we append new line to the Orders_list 
    if  order_id_exists=="true" and item_id_exists=="false":   
            new_order_line["item_id"]=item_id
            new_order_line["item_quantity"]=qty  
            print(type( Orders_list))
            Orders_list.append(new_order_line) 
            return True

def addOneMoreItemLineToRandomExistingOrders(Orders_list):
                                                                        # list of all item_id in the same order
    numb_addtnl_items=int(len(Orders_list)/2)                                         #number of additions
    print("Will be adding: "+str(numb_addtnl_items)+" items")
    
    order_id=0
    unique_order_ids=[]    #iterating through Orders_list and collecting all unique order_id's into unique_order_ids[]
    for order in Orders_list:
        order_id=order["order_id"]
        if order_id not in unique_order_ids:##keyError
            unique_order_ids.append(order_id)  ##
    print("Unique order_id's: ")
    for item in unique_order_ids:
        print(item)
    number_of_unique_order_ids=len(unique_order_ids)
   
    for i in range(1, numb_addtnl_items):
        items_list=[] 
        new_order_line={}
        new_order_line["order_id"]=random.randint(1,number_of_unique_order_ids)    #order_id                                  
        new_order_line["item_id"]=fake.random_element(elements=(2,2,2,2,4,4,4,4,1,1,1,3,3,3,5,5,6,7,7)) #item_id
        new_order_line["item_quantity"]=fake.random_element(elements=(1,1,1,1,1,1,1,1,2,2,2,2,2,3,3,4,5,6)) #item_qty
        item_id_found="false"
                                                                                                              #Verifying if order with generated order_id already contains item with genrated item_id.
                                                                                                           #If yes,we'll be increasing qty.If no,we'll be adding new line order  
        
        for order in Orders_list:  
                if order["order_id"] == new_order_line["order_id"]:
                    items_list.append(order["item_id"])
                    #order_id_found="true" 
                    
        for item in items_list:
                if item==new_order_line["item_id"]:
                    item_id_found="true"
                    
        if item_id_found=="true":
                print("Order with order_id "+ str(new_order_line["order_id"])+" already contains item "+str(new_order_line["item_id"]))
                print("Updating quantity of item "+ str(new_order_line["item_id"])+" to "+str(new_order_line["item_quantity"]))
                print("Calling updateItemQuantity ")
                updateItemQuantity(Orders_list,new_order_line["order_id"], new_order_line["item_id"],new_order_line["item_quantity"])
                
        elif item_id_found=="false":
                print("Will add new item "+ str(new_order_line["item_id"])+" to order id "+ str(new_order_line["order_id"]))
                print("Calling addNewItemToExistingOrder ")
                addNewItemToExistingOrder(Orders_list,new_order_line["order_id"], new_order_line["item_id"],new_order_line["item_quantity"])
            
     
def updateItemQuantity(Orders_list,order_id, item_id, update_to_quantity):
    print("inside updateItemQuantity ")
    items_list=[] 
    order_id_exists="false"
    item_id_exists="false"
    
    #verifying that this order with order_id was already created
    for order in Orders_list:
        if order["order_id"] == order_id:
            items_list.append(order["item_id"])
            order_id_exists="true"  
            
    if order_id_exists=="false":
        print("Order with order_id "+ str(order_id)+" was not created yet")
        return False

    
    #now lets verify that order with order_id already have item with requested item_id 
    elif order_id_exists=="true":
        for item_ in items_list:
            if item_==item_id:
                item_id_exists="true"               
     
    if item_id_exists=="false":
        print("Order with order_id "+ str(order_id)+" doesn't contains item "+str(item_id))
        return False
    
    for order in Orders_list:
        # Assuming 'order_id' and 'item_id' are keys in the dictionaries
        if order["order_id"] == order_id and order["item_id"] == item_id:
            order["item_quantity"] =  update_to_quantity
            return True  # Return True to indicate the update was successful     




In [ ]:

orders=pd.read_csv(r'C:\Users\SOFYA\OneDrive\Desktop\SQL learning resources\cafe chain project\cafe V2\Tables_exported_data\orders.csv')
Orders_list=orders.to_dict(orient="records")

customers=pd.read_csv(r'C:\Users\SOFYA\OneDrive\Desktop\SQL learning resources\cafe chain project\cafe V2\Tables_exported_data\customer.csv')
Customers_list=customers.to_dict(orient="records")

payments=pd.read_csv(r'C:\Users\SOFYA\OneDrive\Desktop\SQL learning resources\cafe chain project\cafe V2\Tables_exported_data\payment.csv')
Payments_list=payments.to_dict(orient="records")


   
addMoreOrdersToOrdersTable(Orders_list,10000)

orderDF=pd.DataFrame(Orders_list)
#order_id_starting_1000 = orderedDF[orderedDF["order_id"] >= 10000]
order_id_starting_1000 =orderDF.query('(order_id>=1000)')
order_id_starting_1000.to_csv('adtnl_10000_payment_ids_for_orders.csv',index=False)




paymentdDF=pd.DataFrame(Payments_list)
payment_id_starting_1000 =paymentdDF.query('(payment_id>=1000)')
payment_id_starting_1000.to_csv('adtnl_10000_records_for_payment.csv',index=False)





782
Inside addNewRecordToPaymentsTable
payment for order_id _ was created
692
Inside addNewRecordToPaymentsTable
payment for order_id _ was created
537
Inside addNewRecordToPaymentsTable
payment for order_id _ was created
544
Inside addNewRecordToPaymentsTable
payment for order_id _ was created
788
Inside addNewRecordToPaymentsTable
payment for order_id _ was created
689
Inside addNewRecordToPaymentsTable
payment for order_id _ was created
103
Inside addNewRecordToPaymentsTable
payment for order_id _ was created
237
Inside addNewRecordToPaymentsTable
payment for order_id _ was created
812
Inside addNewRecordToPaymentsTable
payment for order_id _ was created
198
Inside addNewRecordToPaymentsTable
payment for order_id _ was created
347
Inside addNewRecordToPaymentsTable
payment for order_id _ was created
807
Inside addNewRecordToPaymentsTable
payment for order_id _ was created
713
Inside addNewRecordToPaymentsTable
payment for order_id _ was created
562
Inside addNewRecordToPaymentsTable

In [50]:

# orderdDF=pd.DataFrame(Orders_list)
# orderdDF.to_csv('orders_augmented.csv',index=False)



